[Source kernel](https://github.com/fastai/courses/blob/master/deeplearning1/nbs/lesson3.ipynb)

In [1]:
import os, json, sys
from glob import glob
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
sys.path.insert(0, './../utils')
from importlib import reload
import utils; reload(utils)
from utils import *

Using Theano backend.


In [3]:
path = '../dogs_vs_cats/intermediate/'
model_path = path + 'models/'

if not os.path.exists(model_path): 
    os.mkdir(model_path)

batch_size=4

# Training a better model

Validation set is constant and we dont need to care about. So that, no need to do image augmentation for validation set

## Are we underfitting?

Validation accuracy > training accuracy

## Removing dropout

Our high level approach here will be to start with our fine-tuned cats vs dogs model (with dropout), then fine-tune all the dense layers, after removing dropout from them. The steps we will take are:

- Re-create and load our modified VGG model with binary dependent (i.e. dogs v cats)
- Split the model between the convolutional (conv) layers and the dense layers
- Pre-calculate the output of the conv layers, so that we don't have to redundently re-calculate them on every epoch
- Create a new model with just the dense layers, and dropout p set to zero
- Train this new model using the output of the conv layers as training data.

As before we need to start with a working model, so let's bring in our working VGG 16 model and change it to predict our binary dependent...

In [4]:
model = vgg_ft(2)

...and load our fine-tuned weights.

In [5]:
model.load_weights(model_path+'finetune3.h5')

OSError: Unable to open file (Unable to open file: name = '../dogs_vs_cats/intermediate/models/finetune3.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

We're going to be training a number of iterations without dropout, so it would be best for us to pre-calculate the input to the **fully connected layers** - i.e. the Flatten() layer. We'll start by finding this layer in our model, and creating a new model that contains just the layers up to and including this layer:

In [ ]:
layers = model.layers

In [ ]:
last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Convolution2D][-1]

In [ ]:
last_conv_idx

In [ ]:
layers[last_conv_idx]

In [ ]:
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)
# Dense layers - also known as fully connected or 'FC' layers
fc_layers = layers[last_conv_idx+1:]

Now we can use the exact same approach to creating features as we used when we created the linear model from the imagenet predictions in the last lesson - it's only the model that has changed. As you're seeing, there's a fairly small number of "recipes" that can get us a long way!

In [ ]:
batches = get_batches(path+'train', shuffle=False, batch_size=batch_size)
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)
"""
Validation set is constant and we dont need to care about. 
So that, no need to do image augmentation for validation set
"""
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [ ]:
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)

In [ ]:
trn_features = conv_model.predict_generator(batches, batches.nb_sample)

In [ ]:
save_array(model_path + 'train_convlayer_features.bc', trn_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In [ ]:
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')

In [ ]:
trn_features.shape

For our new fully connected model, we'll create it using the exact same architecture as the last layers of VGG 16, so that we can conveniently copy pre-trained weights over from that model. However, we'll set the dropout layer's p values to zero, so as to effectively remove dropout.

In [ ]:
# Copy the weights from the pre-trained model.
# NB: Since we're removing dropout, we want to half the weights
def proc_wgts(layer): return [o/2 for o in layer.get_weights()]

In [ ]:
# Such a finely tuned model needs to be updated very slowly!
opt = RMSprop(lr=0.00001, rho=0.7)

In [ ]:
def get_fc_model():
    model = Sequential([
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.),
        Dense(4096, activation='relu'),
        Dropout(0.),
        Dense(2, activation='softmax')
        ])

    for l1,l2 in zip(model.layers, fc_layers): l1.set_weights(proc_wgts(l2))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
fc_model = get_fc_model()

In [ ]:
fc_model.fit(trn_features, trn_labels, nb_epoch=8, 
             batch_size=batch_size, validation_data=(val_features, val_labels))

In [ ]:
fc_model.save_weights(model_path+'no_dropout.h5')

In [ ]:
fc_model.load_weights(model_path+'no_dropout.h5')

# Reducing overfitting

- Now that we've gotten the model to overfit, we can take a number of steps to reduce this.
- We should start with overfit model and then reduce overfitting

## Approaches to reducing overfitting

We do **NOT** necessarily need to rely on dropout or other regularization approaches to reduce overfitting. There are other techniques we should try first, since regularlization, by definition, biases our model towards simplicity - which we only want to do if we know that's necessary. This is the order that we recommend using for reducing overfitting (more details about each in a moment):

- Add more data
- Use data augmentation
- Use architectures that generalize well
- Add regularization
- Reduce architecture complexity.

We'll assume that you've already collected as much data as you can, so step (1) isn't relevant (this is true for most Kaggle competitions, for instance). So the next step (2) is data augmentation. This refers to creating additional synthetic data, based on reasonable modifications of your input data. For images, this is likely to involve one or more of: flipping, rotation, zooming, cropping, panning, minor color changes.

Which types of augmentation are appropriate depends on your data. For regular photos, for instance, you'll want to use horizontal flipping, but not vertical flipping (since an upside down car is much less common than a car the right way up, for instance!)

We recommend always using at least some light data augmentation, unless you have so much data that your model will never see the same input twice.

## Data augmentation

### About data augmentation

### Adding data augmentation

## Batch normalization

### About batch norm

### Adding batch norm